# Zuoz - Bever

## Allgemeine Angaben

|   |   |
| --- | --- |
| Start | Zuoz |
| Ziel | Bever |
| Strecke | 13.45 km |
| Aufstieg | 370 m |
| Abstieg | 350 m |
| Marschzeit | 3 h 30 |

In [1]:
import folium
import gpxpy

::: {.content-visible when-format="html"}

In [2]:
import os
from IPython.display import Image, display

# Prüfen, ob PDF-Format
is_pdf = os.environ.get('QUARTO_PROJECT_OUTPUT_FORMAT', '') == 'pdf'

if is_pdf:
    # Für PDF: Verwende das vorhandene PNG
    display(Image(filename='../../../images/250617_map_output.png'))
else:
    # Für HTML: Interaktive Folium-Karte erstellen
    import folium
    import gpxpy
    
    center = [46.576157, 9.919649]
    m = folium.Map(location=center,
                   zoom_start=13,
                   tiles=None
                   )
    
    folium.TileLayer('OpenStreetMap', name='OpenStreetMap (Standard)').add_to(m)
    
    folium.raster_layers.WmsTileLayer(
        url='https://wms.geo.admin.ch/',
        layers='ch.swisstopo.pixelkarte-farbe',
        fmt='image/png',
        name='Swisstopo',
        attr='&copy; <a href="https://www.swisstopo.admin.ch/">swisstopo</a>',
        overlay=False,
        control=True
    ).add_to(m)
    
    # GPX-Verarbeitung
    gpx_path = '250617_zuoz.gpx'
    if os.path.exists(gpx_path):
        with open(gpx_path, 'r') as gpx_file:
            gpx = gpxpy.parse(gpx_file)
    
        gpx_geojson = {
            'type': 'FeatureCollection',
            'features': []
        }
    
        for track in gpx.tracks:
            for segment in track.segments:
                coordinates = [[point.longitude, point.latitude] for point in segment.points]
                
                gpx_geojson['features'].append({
                    'type': 'Feature',
                    'geometry': {
                        'type': 'LineString',
                        'coordinates': coordinates
                    },
                    'properties': {
                        'name': 'GPX Track Segment'
                    }
                })
    
        geojson_layer = folium.GeoJson(
            gpx_geojson,
            name='Zuoz - Bever Kunstwanderung',
            style_function=lambda x: {
                'color': 'red',
                'weight': 3,
                'opacity': 0.7
            }
        ).add_to(m)
        
        m.fit_bounds(geojson_layer.get_bounds())
    
    folium.LayerControl().add_to(m)
    display(m)


:::

::: {.content-visible when-format="pdf"}
![](../../../images/250617_map_output.png)
:::

In [3]:
#| echo: false
#| output: false

import gpxpy
import matplotlib.pyplot as plt
import os

# GPX-Datei laden
gpx_path = '250617_zuoz.gpx'

if os.path.exists(gpx_path):
    with open(gpx_path, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
    
    # Daten extrahieren
    distances = []
    elevations = []
    total_distance = 0
    
    for track in gpx.tracks:
        for segment in track.segments:
            for i, point in enumerate(segment.points):
                if i == 0:
                    distances.append(0)
                else:
                    prev_point = segment.points[i-1]
                    distance = point.distance_2d(prev_point)
                    total_distance += distance
                    distances.append(total_distance / 1000)
                elevations.append(point.elevation)
    
    # Höhenprofil erstellen
    plt.figure(figsize=(12, 4))
    plt.plot(distances, elevations, linewidth=2, color='#d62728')
    plt.fill_between(distances, elevations, alpha=0.3, color='#d62728')
    
    plt.xlabel('Distanz (km)', fontsize=12)
    plt.ylabel('Höhe (m ü. M.)', fontsize=12)
    plt.title('Höhenprofil', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    # Statistiken
    min_elevation = min(elevations)
    max_elevation = max(elevations)
    total_ascent = sum(elevations[i] - elevations[i-1] for i in range(1, len(elevations)) if elevations[i] > elevations[i-1])
    total_descent = sum(elevations[i-1] - elevations[i] for i in range(1, len(elevations)) if elevations[i] < elevations[i-1])
    
    # Y-Achse anpassen: 200m unter dem tiefsten Punkt
    plt.ylim(min_elevation - 200, max_elevation + 50)
    
    stats_text = f'Distanz: {total_distance/1000:.2f} km | Min: {min_elevation:.0f} m | Max: {max_elevation:.0f} m | ↑ {total_ascent:.0f} m | ↓ {total_descent:.0f} m'
    plt.text(0.5, 0.02, stats_text, transform=plt.gca().transAxes,
             ha='center', fontsize=10,
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.savefig('../../../images/250617_elevation_profile.png', dpi=150, bbox_inches='tight')
    plt.close()


In [4]:
#| echo: false
#| output: false

import qrcode

# Der Swisstopo-Link (mit GPX-Track)
swisstopo_url = "https://map.geo.admin.ch/#/map?lang=de&center=2790156.34,1161326.78&z=6&bgLayer=ch.swisstopo.pixelkarte-farbe&topic=ech&layers=GPX|https://raw.githubusercontent.com/Jacques-Mock-Schindler/wanderalbum/main/files/2025/250617_oberengadin/250617_zuoz.gpx"

# QR-Code erstellen
qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4,
)

qr.add_data(swisstopo_url)
qr.make(fit=True)

# Als Bild speichern
img = qr.make_image(fill_color="black", back_color="white")
img.save("../../../images/250617_qr_tag.png")


![](../../../images/250617_elevation_profile.png)


[Die Karte auf der Website von swisstopo.ch
öffnen.](https://map.geo.admin.ch/#/map?lang=de&center=2790156.34,1161326.78&z=6&bgLayer=ch.swisstopo.pixelkarte-farbe&topic=ech&layers=GPX|https://raw.githubusercontent.com/Jacques-Mock-Schindler/wanderalbum/main/files/2025/250617_oberengadin/250617_zuoz.gpx)

::: {.content-visible when-format="pdf"}
![](../../../images/250617_qr_tag.png){width=3cm height=3cm}
:::



::: {.content-visible when-format="html"}

## Bilder

![Die Wanderung startet am Bahnhof Zuoz. Eigentlicher Ausgagnspunkt ist
allerdings das Hotel Castell. Dessen führer Eigentümer mit der
Kunstsammlung, um die es auf dieser Wanderung neben der Oberengadiner
Landschaft geht, gestartet hat.](../../../images/250617_hotel_castell.jpg)

![Die Erste Installation, welche man bei der Ankfunft beim Hotel Castell
sieht, ist *Skyspace Piz Uter* von James Turell. Die Skulptur ist
begehbar. Wenn man sich im Innern dem Eingang gegnüber auf die der Wand
entlanglaufende Bank setzt, sieht man durch den Eingang genau auf den
Piz Uter.](../../../images/250617_skyspace_piz_uter.jpg)


![Der Piz Uter durch den Eingang gesehen.](../../../images/250617_piz_uter.jpg)

![Etwas weniger weit sichtbar, aber nicht minder originell ist das
«Felsenbad» von Tadashi Kawamata.](../../../images/250617_felsenbad.jpg)

::: {.content-visible when-format="html"}
![Roman Signers «Wasserfenster» verändert die Wahrnehmung der Bergwelt.](../../../images/250617_wasserfenster.gif)
:::

::: {.content-visible when-format="pdf"}
![Roman Signers «Wasserfenster» verändert die Wahrnehmung der Bergwelt.](../../../images/250617_wasserfenster_static.jpg)
:::

![In La Punt kann im Hotel Krone der Schlüssel für die meines Erachtens orginellste Installation abgeholt werden.  Der Versuch, eines Oberengadiners, nach China auszuwandern.](../../../images/250617_auswanderer.jpg)



![Der Auswanderer ist die letzte Installation auf der Wanderung. Anschliessend geht es an der linken Talflanke entlang nach Bever.](../../../images/250617_Landschaft_vor_Bever.jpg)

:::